<a href="https://colab.research.google.com/github/amilkh/cs230-fer/blob/master/fer2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip3 install tensorflow-gpu==2.0  #unnecessary on aws if using conda_tensorflow2_p36

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn.metrics as sm

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, InputLayer, MaxPool2D

%matplotlib inline

In [10]:
data = pd.read_csv('fer2013/fer2013.csv')

#print('Number of samples in the dataset: ', data.shape[0])
emotion_cat = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}

# See the target distribution (check for imbalance)
#target_counts = data['emotion'].value_counts().reset_index(drop=False)
#target_counts.columns = ['emotion', 'number_samples']
#target_counts['emotion'] = target_counts['emotion'].map(emotion_cat)
#target_counts

# Transform images from strings to lists of integers. TODO: use an array cast
data['pixels'] = data['pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])

In [11]:
# Select randomly 10 images
#random_seed = 1
#data_sample = data.sample(10, random_state=random_seed)
#f, axarr = plt.subplots(2, 5, figsize=(20, 10))

#i, j = 0, 0
#for idx, row in data_sample.iterrows():
#    img = np.array(row['pixels']).reshape(48,48)
#    axarr[i,j].imshow(img, cmap='gray')
#    axarr[i,j].set_title(emotion_cat[row['emotion']])
#    if j==4:
#        i += 1
#        j = 0
#    else:
#        j += 1

In [14]:
data_train = data[data['Usage']=='Training']
size_train = data_train.shape[0]
#print('Number samples in the training dataset: ', size_train)

data_dev = data[data['Usage']=='PublicTest']
size_dev = data_dev.shape[0]
#print('Number samples in the development dataset: ', size_dev)

# Retrieve train input and target
X_train, y_train = data_train['pixels'].tolist(), data_train['emotion'].values
# Reshape images to 4D (num_samples, width, height, num_channels)
X_train = np.array(X_train, dtype='float32').reshape(-1,48,48,1)
# Normalize images with max (the maximum pixel intensity is 255)
X_train = X_train/255.0

# Retrieve dev input and target
X_dev, y_dev = data_dev['pixels'].tolist(), data_dev['emotion'].values
# Reshape images to 4D (num_samples, width, height, num_channels)
X_dev = np.array(X_dev, dtype='float32').reshape(-1,48,48,1)
# Normalize images with max
X_dev = X_dev/255.0

In [15]:
model = tf.keras.models.Sequential(   
    [
     InputLayer(input_shape=(48,48,1),name="input"),
     Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',name="conv1"),
     Conv2D(filters=32,kernel_size=3,activation='relu',padding='same',name="conv2"),
     MaxPool2D(pool_size=(2,2),name="maxpool1"),
     Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',name="conv3"),
     Conv2D(filters=64,kernel_size=3,activation='relu',padding='same',name="conv4"),
     Flatten(),
     Dense(1024,input_shape=(24*24*64,1),activation='relu',name='fc1'),
     Dense(7,input_shape=(1024,1),activation='softmax',name='fc-softmax')
    ]
)

In [16]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 48, 48, 32)        320       
_________________________________________________________________
conv2 (Conv2D)               (None, 48, 48, 32)        9248      
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 24, 24, 32)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 24, 24, 64)        18496     
_________________________________________________________________
conv4 (Conv2D)               (None, 24, 24, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 1024)              3

In [18]:
model.fit(X_train,y_train,batch_size=32,epochs=10)


Train on 28709 samples
Epoch 1/10
28709/28709 [==============================] - 92s 3ms/sample - loss: 1.5997 - accuracy: 0.3674
Epoch 2/10
28709/28709 [==============================] - 30s 1ms/sample - loss: 1.2936 - accuracy: 0.5067
Epoch 3/10
28709/28709 [==============================] - 30s 1ms/sample - loss: 0.9818 - accuracy: 0.6314
Epoch 4/10
28709/28709 [==============================] - 30s 1ms/sample - loss: 0.5055 - accuracy: 0.8225
Epoch 5/10
28709/28709 [==============================] - 30s 1ms/sample - loss: 0.1582 - accuracy: 0.9560
Epoch 6/10
28709/28709 [==============================] - 30s 1ms/sample - loss: 0.0822 - accuracy: 0.9833
Epoch 7/10
28709/28709 [==============================] - 30s 1ms/sample - loss: 0.0609 - accuracy: 0.9892
Epoch 8/10
28709/28709 [==============================] - 30s 1ms/sample - loss: 0.0513 - accuracy: 0.9904
Epoch 9/10
28709/28709 [==============================] - 30s 1ms/sample - loss: 0.0574 - accuracy: 0.9886
Epoch 10/10
28

In [19]:
y_hat_dev = np.argmax(model.predict(X_dev), axis=1)

In [20]:
import sklearn.metrics as sm
sm.accuracy_score(y_hat_dev, y_dev)

0.5182502089718585